In [36]:
#importing essential libraries for pyspark and sql
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
import mysql.connector
import pandas as pd

#create spark session
spark = SparkSession.builder.config("spark.jars", "/home/hduser/mysql-connector-java-5.1.47/mysql-connector-java-5.1.47.jar") \
    .master("local").appName("ETL to SQL").getOrCreate()
spark

In [37]:
#connecting to localhost sql database
 
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database ='etl'
)
 
# Creating an instance of 'cursor' class
# which is used to execute the 'SQL'
# statements in 'Python'
cursor = mydb.cursor()
 
# Show database tables
cursor.execute("SHOW TABLES")
 
for x in cursor:
  print(x)

#cursor.execute("INSERT INTO crypto (Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap) values('test',0,0,0,0,0,0,0,0)")
#mydb.commit()

('bitcoin',)
('bitcoin1',)
('bitcoin2',)
('cardano',)
('crypto',)
('ethereum',)
('ethereum1',)
('newcrypto',)
('newcrypto1',)
('polkadot',)


In [38]:
def sql_upload(df,table):
    pd_dataframe=df.toPandas()
    cursor.execute("CREATE IF NOT EXISTS "+table+" ( `id` INT NOT NULL AUTO_INCREMENT , `date` DATETIME NOT NULL , `high` DOUBLE NOT NULL , `low` DOUBLE NOT NULL , `open` DOUBLE NOT NULL , `close` DOUBLE NOT NULL , PRIMARY KEY (`id`))")
    for index, row in pd_dataframe.iterrows():
        cursor.execute("INSERT INTO "+table+" (date,high,low,close) values(%s,%s,%s,%s)",(row['date'],row['high'],row['low'],row['close']))
        mydb.commit()
    print("Successfully uploaded to sql table")

In [39]:
btc_df = spark.read.csv('csv\\ETHUSDT_Binance_futures_2022_minute.csv', header=True, sep=",").cache()
btc_df.show()

+-----------+--------------+--------+-------+-------+-------+-------+----------+-----------+----------+
|       unix|          date|  symbol|   open|   high|    low|  close|Volume ETH|Volume USDT|tradecount|
+-----------+--------------+--------+-------+-------+-------+-------+----------+-----------+----------+
|1.66251E+12| 9/7/2022 0:16|ETH/USDT| 1561.5| 1561.5|1555.82|1556.24|  9295.904|14489151.21|      5017|
|1.66251E+12| 9/7/2022 0:15|ETH/USDT|1560.87|1562.76|1560.58|1561.51|  3613.058|5641929.021|      2574|
|1.66251E+12| 9/7/2022 0:14|ETH/USDT|1560.98|   1562| 1560.5|1560.88|   1737.46|2712354.052|      1905|
|1.66251E+12| 9/7/2022 0:13|ETH/USDT|1561.09|1561.98|1560.01|1560.99|  5846.397|9123574.813|      3255|
|1.66251E+12| 9/7/2022 0:12|ETH/USDT|1562.67|1562.67|1561.08|1561.09|  4170.106|6512540.148|      3042|
|1.66251E+12| 9/7/2022 0:11|ETH/USDT|1564.26|1564.27|1562.23|1562.66|  3922.347|6131046.954|      2947|
|1.66251E+12| 9/7/2022 0:10|ETH/USDT| 1566.1|1566.11|1563.76|156

In [40]:
btc_df['close']

Column<'close'>

In [41]:
sql_upload(btc_df,"ethereum1")
print("test")

Successfully uploaded to sql table
test


In [30]:
eth_df = spark.read.csv('csv\\ethereum.csv', header=True, sep=",").cache()
ada_df = spark.read.csv('csv\\cardano.csv', header=True, sep=",").cache()
polka_df = spark.read.csv('csv\\polkadot.csv', header=True, sep=",").cache()

In [31]:
sql_upload(eth_df,"ethereum")
sql_upload(ada_df,"cardano")
sql_upload(polka_df,"polkadot")

Successfully uploaded to sql table
Successfully uploaded to sql table
Successfully uploaded to sql table
